In [2]:
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

### Data loading

In [3]:
raw_data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [4]:
raw_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
raw_data.info() # A lor of columns are objects - so they are either non-converted to a numbers or there are a lot of cat variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [6]:
raw_data.describe() # not much help here

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [7]:
# try convert to numeric and nothing good will happen
for c in raw_data.columns:
    raw_data[c] = pd.to_numeric(raw_data[c], errors='ignore')

In [8]:
for c in raw_data.columns:
    print(c, raw_data[c].drop_duplicates().count())

customerID 7043
gender 2
SeniorCitizen 2
Partner 2
Dependents 2
tenure 73
PhoneService 2
MultipleLines 3
InternetService 3
OnlineSecurity 3
OnlineBackup 3
DeviceProtection 3
TechSupport 3
StreamingTV 3
StreamingMovies 3
Contract 3
PaperlessBilling 2
PaymentMethod 4
MonthlyCharges 1585
TotalCharges 6531
Churn 2


In [9]:
data = raw_data.copy()
# we don't need customer_id column
data.drop(columns=['customerID'], inplace=True)

In [10]:
# just go for a columns one by one
data['gender'].value_counts()

Male      3555
Female    3488
Name: gender, dtype: int64

In [11]:
data['gender'] = data['gender'].apply(lambda x: 0 if x == 'Male' else 1)

In [12]:
# just to replace every Yes/No in once for all columns with just these two values
for col in data.columns:
    if data[col].dtype == 'object' and len(data[col].value_counts().index)==2:
        data[col] = data[col].replace('No', 0).replace('Yes',1)

In [13]:
data.info() # looks better

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   int64  
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   int64  
 3   Dependents        7043 non-null   int64  
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   int64  
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   int64  
 16  PaymentMethod     7043 non-null   object 


In [14]:
for col in data.columns:
    if data[col].dtype == 'object':
        print(col)
        print(data[col].value_counts())

MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64
InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64
OnlineSecurity
No                     3498
Yes                    2019
No internet service    1526
Name: OnlineSecurity, dtype: int64
OnlineBackup
No                     3088
Yes                    2429
No internet service    1526
Name: OnlineBackup, dtype: int64
DeviceProtection
No                     3095
Yes                    2422
No internet service    1526
Name: DeviceProtection, dtype: int64
TechSupport
No                     3473
Yes                    2044
No internet service    1526
Name: TechSupport, dtype: int64
StreamingTV
No                     2810
Yes                    2707
No internet service    1526
Name: StreamingTV, dtype: int64
StreamingMovies
No                     2785
Yes                    2732
No internet service    1526
Name: S

In [15]:
# I would like to add two columns to represent if the subscriber has Internet/phone services
data['HasPhoneService'] = [0  if i == 'No phone service' else 1 for i in data['MultipleLines']]
data['HasInternetService'] = [0  if i == 'No' else 1 for i in data['InternetService']]

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gender              7043 non-null   int64  
 1   SeniorCitizen       7043 non-null   int64  
 2   Partner             7043 non-null   int64  
 3   Dependents          7043 non-null   int64  
 4   tenure              7043 non-null   int64  
 5   PhoneService        7043 non-null   int64  
 6   MultipleLines       7043 non-null   object 
 7   InternetService     7043 non-null   object 
 8   OnlineSecurity      7043 non-null   object 
 9   OnlineBackup        7043 non-null   object 
 10  DeviceProtection    7043 non-null   object 
 11  TechSupport         7043 non-null   object 
 12  StreamingTV         7043 non-null   object 
 13  StreamingMovies     7043 non-null   object 
 14  Contract            7043 non-null   object 
 15  PaperlessBilling    7043 non-null   int64  
 16  Paymen

In [17]:
print(data[data['HasPhoneService'] != data['PhoneService']]) # we already have the column to represent if the user has phone service, so let's just drop duplicated

data.drop(columns=['PhoneService'], inplace=True)

Empty DataFrame
Columns: [gender, SeniorCitizen, Partner, Dependents, tenure, PhoneService, MultipleLines, InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies, Contract, PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges, Churn, HasPhoneService, HasInternetService]
Index: []

[0 rows x 22 columns]


In [18]:
data.sample(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,HasPhoneService,HasInternetService
2914,0,0,0,0,32,No,DSL,No,No,No,...,Yes,Yes,One year,1,Bank transfer (automatic),64.85,2010.95,0,1,1
806,0,0,0,0,18,No,DSL,No,No,Yes,...,No,No,Month-to-month,0,Credit card (automatic),55.20,1058.1,0,1,1
1484,1,1,1,0,71,No,Fiber optic,No,Yes,Yes,...,Yes,Yes,Two year,1,Electronic check,99.20,7213.75,0,1,1
3279,0,0,1,1,23,No,DSL,Yes,No,No,...,No,No,One year,0,Credit card (automatic),57.20,1423.35,0,1,1
5146,0,0,0,1,11,Yes,Fiber optic,No,No,No,...,No,Yes,Month-to-month,0,Electronic check,86.20,893.2,0,1,1


In [19]:
for col in data.columns:
    if data[col].dtype == 'object' and 'No phone service' in data[col].value_counts().index:
        print(col)
        print(data[col].value_counts())

MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64


In [20]:
data['MultipleLines'] = data['MultipleLines'].apply(lambda x: 1 if x == 'Yes' else 0) # because subs without phone service in general also don't have this specific service

In [21]:
inet_cols = []
for col in data.columns:
    if data[col].dtype == 'object' and 'No internet service' in data[col].value_counts().index:
        print(col)
        print(data[col].value_counts())
        inet_cols.append(col)

OnlineSecurity
No                     3498
Yes                    2019
No internet service    1526
Name: OnlineSecurity, dtype: int64
OnlineBackup
No                     3088
Yes                    2429
No internet service    1526
Name: OnlineBackup, dtype: int64
DeviceProtection
No                     3095
Yes                    2422
No internet service    1526
Name: DeviceProtection, dtype: int64
TechSupport
No                     3473
Yes                    2044
No internet service    1526
Name: TechSupport, dtype: int64
StreamingTV
No                     2810
Yes                    2707
No internet service    1526
Name: StreamingTV, dtype: int64
StreamingMovies
No                     2785
Yes                    2732
No internet service    1526
Name: StreamingMovies, dtype: int64


In [22]:
for c in inet_cols:
    data[c] = data[c].apply(lambda x: 1 if x == 'Yes' else 0) # same for other internet-related services

In [23]:
data.dtypes

gender                  int64
SeniorCitizen           int64
Partner                 int64
Dependents              int64
tenure                  int64
MultipleLines           int64
InternetService        object
OnlineSecurity          int64
OnlineBackup            int64
DeviceProtection        int64
TechSupport             int64
StreamingTV             int64
StreamingMovies         int64
Contract               object
PaperlessBilling        int64
PaymentMethod          object
MonthlyCharges        float64
TotalCharges           object
Churn                   int64
HasPhoneService         int64
HasInternetService      int64
dtype: object

In [24]:
data['InternetService'].value_counts()

Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64

In [25]:
inet_service_dummy = pd.get_dummies(data['InternetService'], prefix='InternetService')
inet_service_dummy.head()

,InternetService_DSL,InternetService_Fiber optic,InternetService_No
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,0,1,0


In [26]:
data['Contract'].value_counts()

Month-to-month    3875
Two year          1695
One year          1473
Name: Contract, dtype: int64

In [27]:
contract_dummy = pd.get_dummies(data['Contract'], prefix='Contract')
contract_dummy.head()

,Contract_Month-to-month,Contract_One year,Contract_Two year
0,1,0,0
1,0,1,0
2,1,0,0
3,0,1,0
4,1,0,0


In [28]:
data['PaymentMethod'].value_counts()

Electronic check             2365
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card (automatic)      1522
Name: PaymentMethod, dtype: int64

In [29]:
payment_dummy = pd.get_dummies(data['PaymentMethod'], prefix='Payment')
payment_dummy.head()

,Payment_Bank transfer (automatic),Payment_Credit card (automatic),Payment_Electronic check,Payment_Mailed check
0,0,0,1,0
1,0,0,0,1
2,0,0,0,1
3,1,0,0,0
4,0,0,1,0


In [31]:
all_dummies = inet_service_dummy.iloc[:,:2].join(contract_dummy).join(payment_dummy)

In [32]:
data = data.join(all_dummies)

In [63]:
# data.drop(columns=['InternetService', 'Contract', 'PaymentMethod'], inplace=True)

In [38]:
data.dtypes

gender                                 int64
SeniorCitizen                          int64
Partner                                int64
Dependents                             int64
tenure                                 int64
MultipleLines                          int64
InternetService                       object
OnlineSecurity                         int64
OnlineBackup                           int64
DeviceProtection                       int64
TechSupport                            int64
StreamingTV                            int64
StreamingMovies                        int64
Contract                              object
PaperlessBilling                       int64
PaymentMethod                         object
MonthlyCharges                       float64
TotalCharges                         float64
Churn                                  int64
HasPhoneService                        int64
HasInternetService                     int64
InternetService_DSL                    uint8
InternetSe

In [37]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

In [39]:
data.isnull().sum()

gender                                0
SeniorCitizen                         0
Partner                               0
Dependents                            0
tenure                                0
MultipleLines                         0
InternetService                       0
OnlineSecurity                        0
OnlineBackup                          0
DeviceProtection                      0
TechSupport                           0
StreamingTV                           0
StreamingMovies                       0
Contract                              0
PaperlessBilling                      0
PaymentMethod                         0
MonthlyCharges                        0
TotalCharges                         11
Churn                                 0
HasPhoneService                       0
HasInternetService                    0
InternetService_DSL                   0
InternetService_Fiber optic           0
Contract_Month-to-month               0
Contract_One year                     0


In [40]:
data = data[~data['TotalCharges'].isna()]

In [41]:
data.isnull().sum()

gender                               0
SeniorCitizen                        0
Partner                              0
Dependents                           0
tenure                               0
MultipleLines                        0
InternetService                      0
OnlineSecurity                       0
OnlineBackup                         0
DeviceProtection                     0
TechSupport                          0
StreamingTV                          0
StreamingMovies                      0
Contract                             0
PaperlessBilling                     0
PaymentMethod                        0
MonthlyCharges                       0
TotalCharges                         0
Churn                                0
HasPhoneService                      0
HasInternetService                   0
InternetService_DSL                  0
InternetService_Fiber optic          0
Contract_Month-to-month              0
Contract_One year                    0
Contract_Two year        

In [49]:
data.to_pickle('cleaned_data.pkl')

In [47]:
import os

In [48]:
os.getcwd()

'D:\\_learning\\PY\\ml_academy\\ml_academy\\task_2'